<a href="https://colab.research.google.com/github/CEhresmann/CEhresmann/blob/master/%D0%A2%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Фанфикшн и тематическое моделирование
## Почему фанфикшн?
Хотя фанфикшн (фанатская литература) появилась еще до появления Интернета, распространяясь в небольших фанатских изданиях, сейчас сложно представить этот вид литературы (некоторые исследователи называют это паралитературой, другие считают разновидностью постфольклора) вне Интернета. Это делает его очень привлекательным для анализа - тексты уже сразу же рождаются в цифровом формате, а также, как правило, снабжены так называемой "шапкой" - метаданными. Эти метаданные сообщают фандом, возрастной рейтинг, направленность (например, Слэш - гомосексуальные отношения). Эти метаданные интересны, так как создаются самими участниками сообщества фикрайтеров.
## Что мы делаем?
Предположим, что мы хотим узнать как связаны часть такой шапки - жанр - с содержанием самих текстов. Для начала нам надо собрать корпус текстов с метаданными. Корпус можно балансировать по разным принципам, в нашем случае мы берем по сто текстов четырех направленностой для каждого из четырех жанров (Ангст, Флафф, Фэнтези и Повседневность), всего 1600 текстов. Важно понимать, что каждый из этих текстов относится и к другим жанрам (как правило все тексты имеют больше одной жанровой метки), в нашем случае важно, что тексты в наших четырех жанрах не пересекаются между собой (то есть, мы исключили тексты, присутсвующие в более чем одном из этих четырех жанров).
## Для чего нужно тематическое моделирование?
Как же атоматически определить содержание этих текстов? Одним из популярных способов для этого является тематическое моделирование. По существу, оно позволяет выделять из текстов темы, с какой-то вероятностью порождающие слова и затем смотреть, с какую вероятностью тексты соотносятся с этими темами. Для нашей задачи мы будем применять тематическое моделирование, основанное на Латентном размещении Дирихле ([подробнее прочитать про это можно здесь](https://sysblok.ru/knowhow/kak-ponjat-o-chem-tekst-ne-chitaja-ego/)). Так как нужные алгоритмы уже имплементированы в Питоне, математика, стоящая за ними, нас беспокоить не будет.

Датасет с текстами можно найти на этом [гугл-диске](https://drive.google.com/drive/folders/1uEnZJDzty2u0h9O1pdhPHcUIzeWss8xI?usp=sharing). Функцию для скачивания и другие материалы можно увидеть по [ссылке](https://github.com/makarfedorov/topic_modeling_paraliterature). Для того, чтобы код заработал, необходимо загрузить датасет к себе на диск и заменить путь к файлу в переменной adress на путь к файлу на своем диске. Следует заметить, что pandas не является стандартной библиотекой для Питона и его следует устанавливать, если вы не работаете в Колабе - здесь эта библиотека уже предустановлена

In [2]:
!pip install pyldavis
import os
import pandas as pd
address = "/content/drive/MyDrive/ficbook_one_file/corpus_fanfic.csv"
fanfic_data = pd.read_csv(adress)
os.chdir("/content")

NameError: name 'adress' is not defined

Посмотрим на наши данные

In [ ]:
fanfic_data.head()

Для начала загрузим токенайзер и пунктуацию из библиотеки **nltk**. Кроме этого нам понадобится список стоп-слов

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import download as nltk_download

nltk_download("punkt")
!wget https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt
with open ("stop_ru.txt", "r") as stop_ru:
    rus_stops = [word.strip() for word in stop_ru.readlines()]
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'
filter = rus_stops + list (punctuation)

Теперь установим лемматизатор, для этой цели подойдет библиотека **pymorphy2**. С помощью нее можно приводить слова в начальную форму (программа может приводить в том числе те слова, которые ей "незнакомы")

In [ ]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

parser = MorphAnalyzer()

Напишем функцию для предобработки текста. Слова приводятся к нижнему регистру, стоп-слова удаляются, далее слова лемматизируются



In [ ]:
def preprocess(input_text):
    """
    Функция для предобработки текста. Слова приводятся к нижнему регистру,
    стоп-слова удаляются, далее слова лемматизируются
    :param input_text: Входной текст для очистки и лемматизации
    :return: Очищенный и лемматизированный текст
    """
    text = input_text.lower()
    tokenized_text = word_tokenize(text)
    clean_text = [word for word in tokenized_text if word not in filter]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in
                       clean_text]

    return lemmatized_text

Теперь можно применить эту функцию к нашему датасету с текстами.Это лучше всего сделать с помощью функции **map** (или **apply**), которая может применить созданную нами функцию к каждому полю с текстом в таблице

In [ ]:
fanfic_data["text_processed"] = fanfic_data["text"].map(preprocess)
fanfic_data.head()

Перед тем как делать собственно тематическое моделирование стоит посмотреть какие фандомы есть в дата-сете. Видно, что, во-первых, там есть отдельные фандомы для книг/фильмов по ним. Во-вторых, есть много разных комбинаций из фандомов франшизы Марвел (отдельный фандом для "Первого мстителя", "Железного человека" и т.д.)  и тому подобного.

In [ ]:
fanfic_data["fandom"].unique()
# Роулинг Джоан «Гарри Поттер»,Гарри Поттер', Железный человек,Мстители,Человек-паук: Возвращение домой,  Вдали от дома', 'Железный человек,Первый мститель,Человек-паук: Возвращение домой,  Вдали от дома
# 'Железный человек,Первый мститель,Доктор Стрэндж,Человек-паук: Возвращение домой,  Вдали от дома',

Все эти фандомы необходимо стандартизовать, иначе их получится слишком много для исследования их связи с темами. Это можно сделать выбрав одну название для группы фандомов, которые мы будем считать единым фандомов, и просто заменяя все название на одно. Например, "Властелин колец", "Толкин Джон Р.Р. «Властелин колец»" и "Хоббит" становятся просто "Властелином Колец". Далее можно посмотреть 10 самых частотных фандомов, которые останутся как есть. Остальные низкочастотные фандомы будут помечены либо как "прочее" (если таковой присутствует отдельно), либо как "кроссовер" (если там несколько фандомов). Важно понимать, что функция для стандартизации фандомов работает только с конкетными фандомами, которые есть в нашем датасете. Для другого датасета пришлось бы делать иную стандартизацию (даже если набор фандомов сильно не отличается, их частотности могут быть другими, а значит, список из 10 самых частотных фандомов может быть другим)

In [ ]:
def standart(fandoms):
    """
    Функция для стандартизации фандомов
    :param fandoms: исходный список фандомов
    :return: стандартизованный фандом
    """
    fandom_dictionary = {"Роулинг Джоан «Гарри Поттер»":"Гарри Поттер",
                         "Первый мститель":"Марвел",
                         "Железный человек":"Марвел",
                         "Человек-паук: Возвращение домой,  Вдали от дома":"Марвел",
                         "Доктор Стрэндж":"Марвел",
                         "Толкин Джон Р.Р. «Властелин колец»":"Властелин Колец",
                         "Тор":"Марвел",
                         "Толкин Джон Р. Р. «Хоббит, или Туда и обратно»":"Властелин Колец",
                         "Капитан Марвел":"Марвел",
                         "Сапковский Анджей «Ведьмак» (Сага о ведьмаке)":"Ведьмак",
                         "The Witcher":"Ведьмак",
                         "Мартин Джордж «Песнь Льда и Пламени»":"Игра Престолов",
                         "Boruto: Naruto Next Generations":"Naruto",
                         "Коллинз Сьюзен «Голодные игры»":"Голодные игры",
                         "Удивительный Человек-паук":"Марвел",
                         "Доктор Стрэндж и тайна Ордена магов":"Марвел",
                         "Фантастические твари":"Гарри Поттер",
                         "Marvel Comics":"Марвел","Тор,Доктор Стрэндж":"Марвел",
                         "Мстители,Доктор Стрэндж":"Марвел",
                         "Дэдпул,Дэдпул":"Марвел",
                         "Черная Пантера":"Марвел", "  Вдали от дома":"Марвел",
                         "Мстители":"Марвел", "Новый Человек-паук":"Марвел",
                         "Человек-паук: Возвращение домой":"Марвел",
                         "Человек-Паук":"Марвел", "Хоббит":"Властелин Колец",
                         "Человек-паук: Через Вселенные":"Марвел",
                         "Человек-паук":"Марвел", "Веном":"Марвел",
                         "Стражи Галактики":"Марвел", "Дэдпул":"Марвел",
                         "Черная вдова":"Марвел", "Халк":"Марвел"}
    sig_list = ["Ориджиналы", "Гарри Поттер", "Марвел",
                "Чудесная божья коровка (Леди Баг и Супер-Кот)",
                "Bangtan Boys (BTS)",
                "Однажды в сказке","Fairy Tail", "Сотня", "Naruto", "Волчонок"]

    standart_fandoms = []
    fandoms = fandoms.split(",")
    for fandom in fandoms:
        if fandom in fandom_dictionary:
            fandom = fandom_dictionary[fandom]
        if fandom in sig_list:
            standart_fandoms.append(fandom)
        else:
            standart_fandoms.append("Прочее")

    final = list(set(standart_fandoms))
    if len(final) > 1:
        final = "Кроссовер"
    else:
        final = final[0]

    return final

Можно посмотреть на количество стандартизированных фандомов и, на всякий случай, сохранить их в отдельный файл

In [ ]:
fanfic_data["standart_fandom"] = fanfic_data["fandom"].apply(standart)
fandoms = fanfic_data["standart_fandom"].value_counts()
print(fandoms)
fandoms.to_csv("fandoms3.csv")

Надо импортировать библиотеку **gensim**. Затем мы создаем словарь для тематического моделирования из лемматизированного текста. После создания словаря лучше всего отфлильтровать те слова, которые встречаются в слишком большом количестве текстов, и те, которые встречаются в слишком маленьком количество текстов. Для этого есть метод **filter_extremes**, который принимает в себя аргументы **no_above** (только слова, которые встречаются не более, чем в указанной доле текстов) и **no_below=20** (слова, которые встречаются не менее чем в указанном количестве текстов). После удаления лищних слов, словарь лучше всего ужать в размерах, убрав пропуски с помощью метода **compactify**.

In [ ]:
import gensim
gensim_dictionary = gensim.corpora.Dictionary(fanfic_data["text_processed"])
gensim_dictionary.filter_extremes(no_above=0.1, no_below=20)
gensim_dictionary.compactify()

print(gensim_dictionary)

**Теперь** создаем корпус в виде "мешка слов" (bag of words)

In [ ]:
corpus = [gensim_dictionary.doc2bow(text)
          for text in fanfic_data['text_processed']]

Теперь можно сделать само тематическое моделирование. Для этого, помимо созданного корпуса и словаря, необходимо указать количество "обходов", которые будет делать алгоритм (чем больше, тем точнее и медленнее сооздаваться будет модель) и количество тем, которые мы хотим выделить. Пока возьмем 20 тем (еще стоит не забыть установить **random_state** на какое-нибудь число - это позволит восстановить результат)

In [ ]:
lda_20 = gensim.models.LdaMulticore(corpus,
                                 num_topics=20,
                                 id2word=gensim_dictionary,
                                 passes=10, random_state=6457)

Тематическое моделирование для 20 тем (с помощью Латентного размещения Дирихле)

In [ ]:
lda_20.print_topics()

fw = open("topics_20.txt", "w", encoding="utf-8")
for topic in lda_20.print_topics():
    fw.write(str(topic))
    print(str(topic))
fw.close()

Для того, чтобы узнать, какое количество топиков оптимально, можно использовать метрики, встроенные в библиотеку **gensim** - например, **c_v** или **c_uci**.Посмотрим, какое значение **c_v** есть дя модели на 20 тем

In [ ]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_20,
                                     texts=fanfic_data["text_processed"],
                                     dictionary=gensim_dictionary,
                                     coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()

print("\nCoherence Score: ", coherence_lda)

Значение метрики важно не только само по себе, но и как сравнение с другими возможными количествами тем. Создадим график, пусть на оси х будут отложено количество тем, а на оси у - соответствующее значение метрики. Для этого сначала надо сделать список значений соответвующей метрики для какого-нибудь промежутка тем (для экономии времени лучше вычислять это не для каждого значения количества тем, вполне можно взять шаг в три темы)

In [ ]:
def coherence_score(dictionary, corpus, texts, max, start=2, step=3,
                    measure="c_uci"):
    """
    Функция вычисляет метрики для оценки тем. моделирования и выводит
    график, где по оси x отложено количество топиков, а по оси y - значение
    метрики
    :param dictionary: словарь для тематического моделирования
    :param corpus: корпус в виде мешка слов
    :param texts: тексты документов
    :param max: максимальное количество топиков
    :param start: стартовое количество топиков
    :param step: промежуток, с которым вычисляются топики
    :param measure: метрика
    """
    coherence_values = []
    for num_topics in range(start, max, step):
        model = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary,
                                           passes=10, num_topics=num_topics,
                                           random_state=6457)
        coherencemodel = CoherenceModel(model=model, texts=texts,
                                        dictionary=dictionary,
                                        coherence=measure)
        coherence_values.append(coherencemodel.get_coherence())
    x = range(start, max, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Number of Topics")
    plt.ylabel(measure + "score")
    plt.legend(("coherence_score"), loc='best')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

coherence_score(dictionary=gensim_dictionary, corpus=corpus, texts=fanfic_data["text_processed"], start=2, max=30, step=3)

In [ ]:
coherence_score(dictionary=gensim_dictionary, corpus=corpus, texts=fanfic_data["text_processed"], start=2, max=30, step=3, measure="c_v")

На обоих графиках виден совпадающий пик примерно на 20 темах. Тем не менее, если посмотреть на визуализацию расстояние между топиками, то видно, что 20 топиков сильно накладываются друг на друга. Далее будет видно, что 10 тем накладываются менее существенно, поэтому дальнейшая часть работы дублируется для 20 и для 10 топиков.

*В* библиотеке **gensim** есть встроенная интерактивная визаулизация расстояния между темами. Можно использовать ее для того, чтобы оценить насколько пересекаются между собой полученные темы

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
vis_20 = gensimvis.prepare(lda_20, corpus, gensim_dictionary)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis_20

Видно, что большинство тем находятся очень близко друг к другу. Возможно, стоит посмотреть на модель с другим количеством тем, например, с 10.

In [ ]:
lda_10 = gensim.models.LdaMulticore(corpus,
                                    num_topics=10,
                                    id2word=gensim_dictionary,
                                    passes=10, random_state=6457)

In [ ]:
vis_10 = gensimvis.prepare(lda_10, corpus, gensim_dictionary)

In [ ]:
vis_10

Темы уже не налезают друг на друга так сильно. Можно использовать обе модели - на 10 и на 20 тем, дальнейшие операции будут для них одинаковы.

Теперь необходимо назначить каждому документу в нашем копусе наиболее подходящую (наиболее вероятную) для него тему. Для удобства лучше разделить тему документа и ее вероятность в разные колонки.

In [ ]:
def get_topic(words, lda):
    """
    Функция назначает документу наиболее вероятный топик
    :param words: лемматизированный текст документа
    :param lda: тематическая модель
    :return: список из наиболее вероятного топика
    и его вероятности
    """
    bag = lda.id2word.doc2bow(words)
    topics = lda.get_document_topics(bag)
    topic_dictionary = {}
    for topic in topics:
        topic_dictionary[topic[1]] = str((topic[0]))
    main_probability = max(topic_dictionary)
    main_topic = topic_dictionary[main_probability]

    return [main_topic, main_probability]

In [ ]:
fanfic_data["lda_20"] = fanfic_data["text_processed"].apply(get_topic,
                                                            lda=lda_20)

In [ ]:
fanfic_data["topic_20"] = fanfic_data["lda_20"].str[0]
fanfic_data["probability_20"] = fanfic_data["lda_20"].str[1]
del fanfic_data["lda_20"]
fanfic_data.head()

In [ ]:
fanfic_data["lda_10"] = fanfic_data["text_processed"].apply(get_topic,
                                                            lda=lda_10)

In [ ]:
fanfic_data["topic_10"] = fanfic_data["lda_10"].str[0]
fanfic_data["probability_10"] = fanfic_data["lda_10"].str[1]
del fanfic_data["lda_10"]

fanfic_data.head()

In [ ]:
fanfic_data.describe()

Теперь можно перейти к самому ответу на вопрос, как связаны жанр и фандом фанфика с его содержанием. Построим график, где по горизонтале отложены жанры,а по вертикали - темы. ДЛя этого импортируем библиотеку **seaborn**

In [ ]:
import seaborn as sns

sns.catplot(x="genre", y="topic_20", kind="swarm", data=fanfic_data, height=5,
            aspect=3)

In [ ]:
sns.catplot(x="genre", y="topic_10", kind="swarm",data=fanfic_data, height=5,
            aspect=3)

Видно, что жанры распределяются по темам более-менее равномерно. Теперь посмотрим, как соотносятся с темами стандартизованные фандомы

In [ ]:
sns.catplot(x="standart_fandom", y="topic_20", kind="strip", data=fanfic_data,
            height=5, aspect=6)

In [ ]:
sns.catplot(x="standart_fandom", y="topic_10", kind="strip", data=fanfic_data,
            height=5, aspect=6)

Видно, что, во-первых, фандом "ориджиналы" и "прочее" (низкочастотные фандомы) распределяются по темам равномернее всего. Остальные фандомы преобладающе связаны с одной-двумя темами. Далее можно посмотрить и связь возрастного рейтинга и направленности фанфиков с темами

In [ ]:
sns.catplot(x="rating", y="topic_20", kind="swarm", data=fanfic_data, height=5,
            aspect=3)

In [ ]:
sns.catplot(x="rating", y="topic_10", kind="swarm", data=fanfic_data, height=5,
            aspect=3)

In [ ]:
sns.catplot(x="romance", y="topic_20", kind="swarm", data=fanfic_data,
            height=5, aspect=3)

Среди тем можно было заметить одну, по всей видимости связанную с жанром "омегаверс". Вот как "Книга фанфиков" определяет этот жанр:“Действие работы происходит в мире, где персонажи распределены на три типа: альфы, беты и омеги, каждый из которых обладает рядом физиологических особенностей, сексуальных пристрастий, определенным положением в социальной иерархии и т.д.”.   А вот тема  среди 10 тем, которая очень на него напоминает:   **('0.021*"альфа" + 0.020*"омег" + 0.019*"альф" + 0.018*"омега" + 0.008*"течка" + 0.007*"бета" + 0.007*"виктор" + 0.004*"крис" + 0.004*"гермиона" + 0.003*"марк"')**. Фанфики этого жанра имееют указание на него среди своих меток. Интересно проверить, действительно ли омегаверс связан с определенной темой (темами). Для того, чтобы этого выяснить, сначала нужно выделить метку Омегаверса в отдельную колонку (присутствует ли она среди меток фанфика или нет)


In [ ]:
def omega(data):
    """
    Функция выдает строку Омегаверс, если Омегаверс присутствует
    в списке тегов и строку Не омегаверс, если отсутствует
    :param data: список тегов
    :return: строка Омегаверс или строка Не омегаверс
    """
    omega = ""
    if "Омегаверс" in data:
        omega = "Омегаверс"
    else:
        omega = "Не омегаверс"
    return omega

In [ ]:
fanfic_data["Omega"] = fanfic_data["tag"].apply(omega)
fanfic_data.head()

Теперь можно построить график и посмотреть, как распределяются фанфики омагеверс и не омегаверс относительно тем

In [ ]:
sns.catplot(x="Omega", y="topic_10", kind="strip", data=fanfic_data, height=5,
            aspect=3)

In [ ]:
sns.catplot(x="Omega", y="topic_20", kind="strip", data=fanfic_data, height=5,
            aspect=3)

Видно, что омегаверс преимущественно связан с несколькими темами. Можно сделать вывод, что, хотя те жанры, которые мы выбрали, мало связаны с конкретными темами, есть жанры, которые с ними связаны. Важно сделать уточнение - если посмотреть на выделившиеся темы, то легко увидеть, насколько много там встречаются имен. В дальнейшем, было бы полезно сделать тематическое моделирование, предварительно удалив именные сущности из текста, но здесь мы пока заниматься этим не будем